# Default networks

There are three default networks in Docker. Let's look at them all.

## Network `none`

Connecting a container to the `none` network completely isolates it from any network interactions. The container will have no network access, preventing communication with other containers or external networks.

### Host to container

In the following cell, there is a simple bash script that allows you to compare the behavior of a container connected to the `bridge` network and a container connected to the `none` network.

At the beginning, a container is started using the `bridge` network. When the command `curl localhost:80` is executed, a basic nginx response is received. 

Then, the network for the container is changed to `none`. As a consequence, when the command `curl localhost:80` is executed again, there is no response. This lack of response is a consequence of using the `none` network for the container.

Overall, this script demonstrates the impact of network selection on the container's ability to communicate and receive responses.

In [2]:
%%bash
docker run --rm -itd\
    -p 80:80 \
    --name network_example\
    nginx &> /dev/null
sleep 2

echo "=====DEFAULT NETWORK====="
echo "-----network name-----"
docker inspect -f \
    '{{range $key, $value := .NetworkSettings.Networks}}{{$key}} {{end}}' \
    network_example
echo "-----curl to nginx-----"
curl -s localhost:80 | head -n 10

docker network disconnect bridge network_example
docker network connect none network_example

echo
echo
echo "=====none NETWORK====="
echo "-----network name-----"
docker inspect -f \
    '{{range $key, $value := .NetworkSettings.Networks}}{{$key}} {{end}}' \
    network_example
echo "-----curl to nginx-----"
curl -s localhost:80 | head -n 10

docker stop network_example &> /dev/null

=====DEFAULT NETWORK=====
-----network name-----
bridge 
-----curl to nginx-----
<!DOCTYPE html>
<html>
<head>
<title>Welcome to nginx!</title>
<style>
html { color-scheme: light dark; }
body { width: 35em; margin: 0 auto;
font-family: Tahoma, Verdana, Arial, sans-serif; }
</style>
</head>


=====none NETWORK=====
-----network name-----
none 
-----curl to nginx-----


### From container to internet

Another feature of the `none` network is that it does not have access to the internet. for instance, you can use the `curl` command to fetch data from any website while the docker container is connected to the `bridge` network. however, if you switch the network to `none`, the `curl` command won't work.

The script demonstrates this scenario. initially, the container is connected to the `bridge` network, and executing the `curl example.com` command within the container successfully fetches the webpage. but after switching the container to the `none` network, the same `curl` command does not produce any output.

In [3]:
%%bash
docker run --rm -itd\
    --name network_example\
    nginx &> /dev/null
sleep 2

echo "=====DEFAULT NETWORK====="
echo "-----network name-----"
docker inspect -f \
    '{{range $key, $value := .NetworkSettings.Networks}}{{$key}} {{end}}' \
    network_example
echo "-----curl from bridge-----"
docker exec network_example curl -s http://example.com/ | head -n 10

docker network disconnect bridge network_example
docker network connect none network_example

echo
echo
echo "=====none NETWORK====="
echo "-----network name-----"
docker inspect -f \
    '{{range $key, $value := .NetworkSettings.Networks}}{{$key}} {{end}}' \
    network_example
echo "-----curl from none-----"
docker exec network_example curl -s http://example.com/ | head -n 10

docker stop network_example &> /dev/null

=====DEFAULT NETWORK=====
-----network name-----
bridge 
-----curl from bridge-----
<!doctype html>
<html>
<head>
    <title>Example Domain</title>

    <meta charset="utf-8" />
    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1" />
    <style type="text/css">
    body {


=====none NETWORK=====
-----network name-----
none 
-----curl from none-----


## Network `host`

Completely removes the container's network isolation, that is, the container shares the network completely with the host.

### Connect to `host`

**Note** you can't connect or disconnect a container to the `host` network. It will cause an error. You can only create containers that are already connected to the `host` network.

So the following cell shows what message you will get when you try to connect or disconnect a container to the `host` network.

In [7]:
%%bash
docker run --rm -itd\
    --name network_example\
    nginx &> /dev/null
sleep 2

docker network disconnect bridge network_example
docker network connect host network_example

docker stop network_example &> /dev/null

Error response from daemon: container cannot be disconnected from host network or connected to host network


### Ports with `host` network

If you are using the `host` network, you don't need to worry about setting ports for the container because it will share all ports with the host (the computer the container is running on).

So, the following code cell starts an nginx container twice without using any specified ports. In the first case, it doesn't specify any network, but in the second case, the `--net host` parameter is used. When `curl localhost:80` is executed in the first case, it doesn't return anything, but in the second case, it returns the nginx welcome page

In [5]:
%%bash
docker run --rm -itd\
    --name network_example\
    nginx &> /dev/null
sleep 2 

echo "=====DEFAULT NETWORK====="
echo "-----network name-----"
docker inspect -f \
    '{{range $key, $value := .NetworkSettings.Networks}}{{$key}} {{end}}' \
    network_example
echo "-----curl to bridge-----"
curl -s localhost:80

docker stop network_example &> /dev/null
sleep 2 &> /dev/null

docker run --rm -itd\
    --name network_example\
    --net host\
    nginx &> /dev/null
sleep 2

echo
echo
echo "=====host NETWORK====="
echo "-----network name-----"
docker inspect -f \
    '{{range $key, $value := .NetworkSettings.Networks}}{{$key}} {{end}}' \
    network_example
echo "-----curl to host-----"
curl -s localhost:80 | head -n 10

docker stop network_example &> /dev/null

=====DEFAULT NETWORK=====
-----network name-----
bridge 
-----curl to bridge-----


=====host NETWORK=====
-----network name-----
host 
-----curl to host-----
<!DOCTYPE html>
<html>
<head>
<title>Welcome to nginx!</title>
<style>
html { color-scheme: light dark; }
body { width: 35em; margin: 0 auto;
font-family: Tahoma, Verdana, Arial, sans-serif; }
</style>
</head>
